# Import python 库 

In [1]:
import mitreattack
from stix2 import MemoryStore
from stix2 import Filter
from stix2 import FileSystemSource


# 加载本地数据

In [2]:
# 通过bundle加载数据
src = MemoryStore()
src.load_from_file("../assets/attck_database/attack-stix-data/enterprise-attack/enterprise-attack.json")

# 加载[移除弃用和撤销的对象]方法

In [3]:
# 移除弃用和撤销的对象
from stix2 import Filter

def remove_revoked_deprecated(stix_objects):
    """Remove any revoked or deprecated objects from queries made to the data source"""
    # Note we use .get() because the property may not be present in the JSON data. The default is False
    # if the property is not set.
    return list(
        filter(
            lambda x: x.get("x_mitre_deprecated", False) is False and x.get("revoked", False) is False,
            stix_objects
        )
    )

#datas = src.query([ Filter("type", "=", "course-of-action") ])
#datas = remove_revoked_deprecated(datas)

# 获取所有Tactic

In [6]:
# 获取所有Tactic
from stix2 import Filter

def getTacticsByMatrix(thesrc):
    tactics = {}
    matrix = thesrc.query([
        Filter('type', '=', 'x-mitre-matrix'),
    ])

    # 丢弃启用数据
    matrix= remove_revoked_deprecated(matrix)

    for i in range(len(matrix)):
        tactics[matrix[i]['name']] = []
        for tactic_id in matrix[i]['tactic_refs']:
            tactics[matrix[i]['name']].append(thesrc.get(tactic_id))

    return tactics

# get tactic layout
data_dict = getTacticsByMatrix(src)

# 提取所有的 "name" 字段和对应的 "external_id" 值
name_external_id_pairs = []
for item in data_dict['Enterprise ATT&CK']:
    name = item['name']
    stix_id = item['id']
    external_id = item['external_references'][0]['external_id']
    description = item['description']
    
    name_external_id_pairs.append((name, external_id,stix_id,description))



In [7]:
# 数据处理，转换成支持Markdown格式的数据

# 打印结果
for name, external_id,stix_id,description in name_external_id_pairs:
    print(f"## [{external_id}]:{name}")
    print(f"> External ID: {external_id}")
    print(f"> STIX ID: {stix_id}")
    print(f"\n \n Description: \n {description}")

## [TA0043]:Reconnaissance
> External ID: TA0043
> STIX ID: x-mitre-tactic--daa4cbb1-b4f4-4723-a824-7f1efd6e0592

 
 Description: 
 The adversary is trying to gather information they can use to plan future operations.

Reconnaissance consists of techniques that involve adversaries actively or passively gathering information that can be used to support targeting. Such information may include details of the victim organization, infrastructure, or staff/personnel. This information can be leveraged by the adversary to aid in other phases of the adversary lifecycle, such as using gathered information to plan and execute Initial Access, to scope and prioritize post-compromise objectives, or to drive and lead further Reconnaissance efforts.
## [TA0042]:Resource Development
> External ID: TA0042
> STIX ID: x-mitre-tactic--d679bca2-e57d-4935-8650-8031c87a4400

 
 Description: 
 The adversary is trying to establish resources they can use to support operations.

Resource Development consists of t

# 获取 techniques 或 sub-techniques

In [ ]:
# 获取 techniques 或 sub-techniques
from stix2 import Filter

def get_techniques_or_subtechniques(thesrc, include="both"):
    """Filter Techniques or Sub-Techniques from ATT&CK Enterprise Domain.
    include argument has three options: "techniques", "subtechniques", or "both"
    depending on the intended behavior."""
    if include == "techniques":
        query_results = thesrc.query([
            Filter('type', '=', 'attack-pattern'),
            Filter('x_mitre_is_subtechnique', '=', False)
        ])
    elif include == "subtechniques":
        query_results = thesrc.query([
            Filter('type', '=', 'attack-pattern'),
            Filter('x_mitre_is_subtechnique', '=', True)
        ])
    elif include == "both":
        query_results = thesrc.query([
            Filter('type', '=', 'attack-pattern')
        ])
    else:
        raise RuntimeError("Unknown option %s!" % include)

    return query_results


data_list = get_techniques_or_subtechniques(src, "both")
data_list = remove_revoked_deprecated(data_list)



# 提取所有的 "name" 字段和对应的 "external_id" 值
result = []
for item in data_list:
    name = item.name
    external_id = item.external_references[0].external_id
    attack_id = item.id
    platforms = item.x_mitre_platforms
    phase_names = [phase.phase_name for phase in item.kill_chain_phases]
    description = item.description
    result.append((name, external_id, attack_id, platforms, phase_names, description))


result = sorted(result, key=lambda x: x[1])


In [ ]:
# 数据处理，转换成支持Markdown格式的数据
# 打印结果
for name, external_id, attack_id, platforms, phase_names, description in result:
    if "." in external_id:
        print(f"### [{external_id}]:{name}")
        print(f"> External ID: {external_id}")
        print(f"> STIX ID: {attack_id}")
        print(f"> Platforms: {platforms}")
        print(f"> Tactics: {phase_names}")
        print(f"\n \n Description: \n {description}")
    else:
        print(f"## [{external_id}]:{name}")
        print(f"> External ID: {external_id}")
        print(f"> STIX ID: {attack_id}")
        print(f"> Platforms: {platforms}")
        print(f"> Tactics: {phase_names}")
        print(f"\n \n Description: \n {description}")


# 通过Tactic获取techniques

In [4]:
# 通过Tactic获取techniques
from stix2 import Filter

def get_tactic_techniques(thesrc, tactic):
    # double checking the kill chain is MITRE ATT&CK
    # note: kill_chain_name is different for other domains:
    #    - enterprise: "mitre-attack"
    #    - mobile: "mitre-mobile-attack"
    #    - ics: "mitre-ics-attack"
    return thesrc.query([
        Filter('type', '=', 'attack-pattern'),
        Filter('kill_chain_phases.phase_name', '=', tactic),
        Filter('kill_chain_phases.kill_chain_name', '=', 'mitre-attack'),
    ])


# use the x_mitre_shortname as argument
data_list = get_tactic_techniques(src, 'impact')
data_list = remove_revoked_deprecated(data_list)


result = []
for item in data_list:
    name = item.name
    external_id = item.external_references[0].external_id
    attack_id = item.id
    result.append((name, external_id, attack_id))


result = sorted(result, key=lambda x: x[1])

In [5]:
# 数据处理，生成 markdown 表格

## 生成表头
print(f"|Tech|Sub-Tech|中文|External ID|")
print(f"|---|---|---|---|")


## 贴入数据
for name, external_id, attack_id in result:
    if "." in external_id:
        print(f"||[{name}](###[{external_id}]:{name})||{external_id}|")
    else:
        print(f"|[{name}](##[{external_id}]:{name})|||{external_id}|")

|Tech|Sub-Tech|中文|External ID|
|---|---|---|---|
|[Data Destruction](##[T1485]:Data Destruction)|||T1485|
|[Data Encrypted for Impact](##[T1486]:Data Encrypted for Impact)|||T1486|
|[Service Stop](##[T1489]:Service Stop)|||T1489|
|[Inhibit System Recovery](##[T1490]:Inhibit System Recovery)|||T1490|
|[Defacement](##[T1491]:Defacement)|||T1491|
||[Internal Defacement](###[T1491.001]:Internal Defacement)||T1491.001|
||[External Defacement](###[T1491.002]:External Defacement)||T1491.002|
|[Firmware Corruption](##[T1495]:Firmware Corruption)|||T1495|
|[Resource Hijacking](##[T1496]:Resource Hijacking)|||T1496|
|[Network Denial of Service](##[T1498]:Network Denial of Service)|||T1498|
||[Direct Network Flood](###[T1498.001]:Direct Network Flood)||T1498.001|
||[Reflection Amplification](###[T1498.002]:Reflection Amplification)||T1498.002|
|[Endpoint Denial of Service](##[T1499]:Endpoint Denial of Service)|||T1499|
||[OS Exhaustion Flood](###[T1499.001]:OS Exhaustion Flood)||T1499.001|
||[Ser